In [1]:
import pandas as pd
import numpy as np
from math import log2
import os
import glob
from tqdm import tqdm  # Import the progress bar library

In [3]:
# Function to calculate Chao-Shen entropy
def chao_shen(q):
    yx = q[q > 0]  # Remove bins with zero counts
    n = np.sum(yx)  # Total count
    p = yx.astype(float) / n  # Observed probabilities
    f1 = np.sum(yx == 1)  # Number of singletons in the sample

    if f1 == n:  # Avoid division by zero when all are singletons
        f1 -= 1

    C = 1 - (f1 / n)  # Estimated sample coverage
    pa = C * p  # Coverage-adjusted probabilities
    la = 1 - (1 - pa) ** n  # Probability of observing each category
    H = -np.sum((pa * np.log2(pa)) / la)  # Chao-Shen entropy

    return H, pa, la

# Function to calculate Chao-Shen transition entropy
def calculate_chao_shen_transition_entropy(raw_matrix, stationary_distribution):
    """
    Calculate entropy with Chao-Shen correction using the raw transition matrix counts.
    """
    chao_shen_total_entropy = 0
    chao_shen_category_entropies = {}

    for i, row in raw_matrix.iterrows():
        counts = row.values.astype(int)  # Use raw counts for Chao-Shen
        if (counts.sum() > 1) & (len(row[row == 0.0]) < (len(row) - 1)):
            H, _, _ = chao_shen(counts)  # Apply Chao-Shen correction
            chao_shen_category_entropies[i] = H
            chao_shen_total_entropy += H * stationary_distribution.get(i, 0)  # Weighted by stationary distribution
    return chao_shen_total_entropy, chao_shen_category_entropies


# Function to calculate transition entropy
def calculate_transition_entropy(matrix, stationary_distribution):
    total_entropy = np.NaN
    category_entropies = {}
    for i, row in matrix.iterrows():
        row_entropy = sum(-p * log2(p) for p in row if p > 0)
        category_entropies[i] = row_entropy
        total_entropy += row_entropy * stationary_distribution.get(i, 0)
    return total_entropy, category_entropies


In [2]:
# Mapping dictionary
mapping = {
    "TaskBuilding_Public": "Task_Building",
    "TaskBuilding_Residential": "Task_Building",
    "Active_Agent": "Agent",
    "Passive_Agent": "Agent",
    "Active_Agent_Face": "Agent",
    "Passive_Agent_Face": "Agent"
}

# Collider list
collider_list = [
    '56_Sa', '39_Sa', '19_Cma', '55_Sa', '25_Cma', '40_Sa', '41_Sa',
    '17_Cma', '47_Sa', '03_Cma', '13_Cma', '24_Cma', '01_Cma', '54_Sa',
    '15_Cma', '29_Sa', '04_Cma', '49_Sa', '30_Sa', '02_Cma', '51_Sa',
    '08_Cma', '28_Cma', '26_Cma', '44_Sa', '06_Cma', '53_Sa', '37_Sa',
    '32_Sa', '20_Cma', '16_Cma', '50_Sa', '34_Sa', '11_Cma', '38_Sa',
    '33_Sa', '12_Cma', '22_Cma', '42_Sa', '05_Cma', '23_Cma', '18_Cma',
    '27_Cma', '45_Sa', '43_Sa', '09_Cma', '31_Sa', '48_Sa', '10_Cma',
    '52_Sa', '07_Cma', '46_Sa', '35_Sa', '36_Sa', '21_Cma', '14_Cma'
]

# Define paths
input_dir = "/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/"
output_trials_dir = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Window/trials_df/"
output_transition_dir = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Window/transition_matrix/"
output_entropy_dir = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Window/entropy_results/Chao_Shen/"

# Ensure output directories exist
os.makedirs(output_trials_dir, exist_ok=True)
os.makedirs(output_transition_dir, exist_ok=True)
os.makedirs(output_entropy_dir, exist_ok=True)

file_paths = glob.glob(os.path.join(input_dir, "*.csv"))


In [4]:
# Process all CSV files
for file_path in tqdm(file_paths, desc="Processing Files", unit="file"):
    # Extract participant ID from the file path
    participant_id = os.path.basename(file_path).split('.')[0]

    # Load the data
    data = pd.read_csv(file_path)
    data['date_seconds'] = pd.to_datetime(data['timeStampDataPointEnd'], unit='s')

    # Filter for the desired gaze events
    data_Reduced = data[data['events'] == -2]

    # Filter and label rows with colliders
    filtered_df = data_Reduced[data_Reduced['names'].isin(collider_list)].copy()
    filtered_df['Occurrence_Order'] = filtered_df.groupby('names').cumcount() + 1

    # Segment data by each occurrence of colliders
    last_processed_time = {}
    trials = []
    trial_window_times = {}  # Dictionary to store window times for each trial
    for index, row in filtered_df.iterrows():
        collider_name = row['names']
        occurrence_time = row['date_seconds']

        if (
            collider_name in last_processed_time 
            and (occurrence_time - last_processed_time[collider_name]).total_seconds() <=30
        ):
            continue

        last_processed_time[collider_name] = occurrence_time

        window_start = max(data_Reduced['date_seconds'].min(), occurrence_time)
        window_end = min(data_Reduced['date_seconds'].max(), occurrence_time + pd.Timedelta(seconds=30))

        trial_segment = data_Reduced[
            (data_Reduced['date_seconds'] >= window_start) &
            (data_Reduced['date_seconds'] <= window_end)
        ].copy()
        trial_id = f"{collider_name}_Trial_{len(trials) + 1}"

        trial_segment['Collider_Name'] = collider_name
        trial_segment['Occurrence_Order'] = len(trials) + 1
        trial_segment['Trial_ID'] = trial_id
        
        trials.append(trial_segment)
        
        # Store the trial's window start and end times
        trial_window_times[trial_id] = (window_start, window_end)

    if trials:
        trials_df = pd.concat(trials, ignore_index=True)
        trials_df.to_csv(os.path.join(output_trials_dir, f"{os.path.basename(file_path).replace('.csv', '_trials.csv')}"), index=False)
    else:
        print(f"No trials found for file: {file_path}")
        continue

    # Initialize a list to store entropy results for this participant
    chao_shen_entropy_results = []

    # Calculate entropy for each trial
    for trial_id, trial_data in trials_df.groupby('Trial_ID'):
        collider_name = trial_data['Collider_Name'].iloc[0]
        occurrence_order = trial_data['Occurrence_Order'].iloc[0]

        trial_data["Mapped_Column_Collider_Categorical"] = trial_data["Collider_CategoricalN"].replace(mapping)
        gaze_sequence = trial_data['Mapped_Column_Collider_Categorical'].reset_index(drop=True)

        categories = gaze_sequence.unique()
        raw_transition_matrix = pd.DataFrame(0, index=categories, columns=categories, dtype=float)

        # Build raw transition matrix
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            raw_transition_matrix.loc[current_category, next_category] += 1

        # Normalize transition matrix rows
        transition_matrix = raw_transition_matrix.div(raw_transition_matrix.sum(axis=1), axis=0).fillna(0)

        # Ensure proper normalization of the stationary distribution
        try:
            eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
            stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
            stationary_distribution /= stationary_distribution.sum()

            if not np.isclose(stationary_distribution.sum(), 1):
                print(f"Warning: Stationary distribution does not sum to 1 (sum={stationary_distribution.sum()}). Normalizing.")
                stationary_distribution /= stationary_distribution.sum()

            stationary_distribution_dict = {categories[i]: stationary_distribution[i] for i in range(len(categories))}
        except Exception as e:
            print(f"Warning: Using fallback stationary distribution due to error: {e}")
            stationary_distribution_dict = {category: 1 / len(categories) for category in categories}

        # Calculate Chao-Shen transition entropy
        chao_shen_overall_transition_entropy, chao_shen_transition_entropy_per_category = calculate_chao_shen_transition_entropy(
            raw_transition_matrix, stationary_distribution_dict
        )

        # Normalize entropies
        num_categories = len(categories)
        normalized_chao_shen_overall_entropy = (
            chao_shen_overall_transition_entropy / np.log2(num_categories) if num_categories > 1 else 0
        )
        normalized_chao_shen_transition_entropy_per_category = {
            category: entropy / np.log2(num_categories) if num_categories > 1 else 0
            for category, entropy in chao_shen_transition_entropy_per_category.items()
        }

        # Store results for this trial
        chao_shen_result = {
            'Trial_ID': trial_id,
            'Collider_Name': collider_name,
            'Occurrence_Order': occurrence_order,
            'Gaze_Sequence_Length': len(gaze_sequence),
            'Chao_Shen_Overall_Transition_Entropy': normalized_chao_shen_overall_entropy,
            'Window_Start': window_start,  
            'Window_End': window_end,     
        }

        # Add per-category entropies to the result
        for category in categories:
            chao_shen_result[f'Chao_Shen_Transition_Entropy_{category}'] = normalized_chao_shen_transition_entropy_per_category.get(
                category, 0
            )
        chao_shen_entropy_results.append(chao_shen_result)

    # Save all results for this participant to a single file
    chao_shen_entropy_df = pd.DataFrame(chao_shen_entropy_results)
    chao_shen_entropy_df.to_csv(os.path.join(output_entropy_dir, f"Chao_Shen_{participant_id}_entropy_results.csv"), index=False)


Processing Files:   1%|▏                      | 1/145 [00:02<05:14,  2.18s/file]

Processing Files:   6%|█▍                     | 9/145 [00:22<05:48,  2.56s/file]

Processing Files:   9%|█▉                    | 13/145 [00:32<05:33,  2.53s/file]

Processing Files:  26%|█████▌                | 37/145 [01:22<03:30,  1.95s/file]

Processing Files:  26%|█████▊                | 38/145 [01:25<03:48,  2.13s/file]

...

Processing Files:  80%|████████████████▊    | 116/145 [04:33<01:11,  2.47s/file]

Processing Files:  91%|███████████████████  | 132/145 [05:11<00:30,  2.34s/file]

Processing Files:  95%|███████████████████▉ | 138/145 [05:22<00:14,  2.11s/file]

Processing Files:  99%|████████████████████▋| 143/145 [05:36<00:05,  2.61s/file]

Processing Files: 100%|█████████████████████| 145/145 [05:41<00:00,  2.35s/file]
